# Kaggle Intermediate ML Dealing with Missing Values

## Again we will use the housing data from Kaggle's competition

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Read the data
X_full = pd.read_csv('./train.csv', index_col='Id')
X_test_full = pd.read_csv('./test.csv', index_col='Id')

# Remove rows from our full dataset with missing target (y). Separate
# target from predictors (X)
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we will use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Create train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [3]:
# View the first couple rows of our training predictors
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [4]:
# See the shape of our training data (number of rows, number of columns)
print(X_train.shape)

# Collect the number of missing values in each column of the training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [5]:
# Create a scoring function to determine how we should deal with missing values

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [6]:
# Approach 1, drop columns with missing data

missing_data_columns = [col for col in X_train.columns
                        if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(missing_data_columns, axis=1)
reduced_X_valid = X_valid.drop(missing_data_columns, axis=1)

In [7]:
# Compute MAE of Approach 1 (dropping columns with missing data)

print('MAE (Drop columns with missing values):')
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [8]:
# Approach 2, use an Imputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputer removed our column names, we need to add them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [9]:
# Print our MAE for Approach 2 (using Imputer)

print('MAE (Imputation):')
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [29]:
# Approach 3, an extension on Imputer where we create new columns keeping track of missing values

# First let's make a copy of our training and test datasets so we don't modify the originals
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Add additional columns to our datasets that have missing values
for col in missing_data_columns:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
# Use our imputed extension datasets for train and validation

X_train_imputed_extension = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
X_valid_imputed_extension = pd.DataFrame(my_imputer.transform(X_valid_plus))

In [26]:
# Print our MAE for Approach 3 (Imputer extension)

print('MAE (Imputation extension:)')
print(score_dataset(X_train_imputed_extension, X_valid_imputed_extension, y_train, y_valid))

MAE (Imputation extension:)
18148.417180365297


In [43]:
# Now let's create a RandomForestRegressor model and see how well we predict

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train_imputed_extension, y_train)

# Since we added columns to X_train_imputed_extension we need to add these to our X_test dataset

for col in missing_data_columns:
    X_test[col + '_was_missing'] = X_test[col].isnull()

X_test_imputed = my_imputer.transform(X_test)
    
preds_valid = model.predict(X_test_imputed)

In [45]:
# Print the first few values of our predictions
print('X_test predictions:', preds_valid)

X_test predictions: [126220.   155131.5  180421.64 ... 155044.5  108288.08 229397.65]
